# Linear Regression from First Principles
## A Complete Mathematical and Computational Exploration

---

### Table of Contents
1. [Setup & Imports](#setup)
2. [Mathematical Theory](#theory)
   - 1-Neuron Linear Model
   - Gradient Derivation for w
   - Gradient Derivation for b
   - Why Gradient Descent Works
   - Backpropagation View
3. [Detailed Proofs](#proofs)
   - Component-wise Gradient Proof
   - 2D Example
4. [Implementation](#implementation)
   - Basic Functions
   - Gradient Computation
   - Training Loop
5. [Experiment](#experiment)
   - Data Generation
   - Model Training
   - Results Visualization
6. [Verification](#verification)
7. [Summary](#summary)

---

<a id='setup'></a>
## 1. Setup & Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

<a id='theory'></a>
## 2. Mathematical Theory

# Linear Regression: Mathematical Derivation

## 1-Neuron Linear Model

We have:
* Data: $x_i \in \mathbb{R}^d$, target $y_i \in \mathbb{R}$
* Parameters: $w \in \mathbb{R}^d$, $b \in \mathbb{R}$
* Prediction:
$$\hat{y}_i = w^\top x_i + b$$

**Loss (MSE over $n$ samples):**
$$L(w,b) = \frac{1}{n}\sum_{i=1}^{n}(\hat{y}_i - y_i)^2 = \frac{1}{n}\sum_{i=1}^{n}(w^\top x_i + b - y_i)^2$$

Define the error:
$$e_i = \hat{y}_i - y_i = w^\top x_i + b - y_i$$

So:
$$L = \frac{1}{n}\sum_{i=1}^{n} e_i^2$$

## Gradient w.r.t. $w$

Use chain rule per-sample. For one term $e_i^2$:
$$\frac{\partial}{\partial w}(e_i^2) = 2e_i\frac{\partial e_i}{\partial w}$$

But:
$$e_i = w^\top x_i + b - y_i \Rightarrow \frac{\partial e_i}{\partial w} = x_i$$

So:
$$\frac{\partial}{\partial w}(e_i^2) = 2e_i x_i$$

Sum and scale by $\frac{1}{n}$:
$$\nabla_w L = \frac{1}{n}\sum_{i=1}^n 2e_i x_i = \frac{2}{n}\sum_{i=1}^n (w^\top x_i + b - y_i)x_i$$

**Vectorized form**: Stack rows of $X\in\mathbb{R}^{n\times d}$, $y,\hat{y}\in\mathbb{R}^{n\times 1}$.

Let $e=\hat{y}-y$. Then:
$$\nabla_w L = \frac{2}{n}X^\top e$$

## Gradient w.r.t. $b$

Again:
$$\frac{\partial}{\partial b}(e_i^2) = 2e_i\frac{\partial e_i}{\partial b}$$

And:
$$\frac{\partial e_i}{\partial b} = 1$$

So:
$$\frac{\partial}{\partial b}(e_i^2) = 2e_i$$

Sum and scale:
$$\frac{\partial L}{\partial b} = \frac{2}{n}\sum_{i=1}^n e_i = \frac{2}{n}\sum_{i=1}^n (\hat{y}_i - y_i)$$

Vectorized:
$$\frac{\partial L}{\partial b} = \frac{2}{n}\mathbf{1}^\top e$$

## Why Gradient Descent Update Works

We move opposite the gradient:
$$w \leftarrow w - \eta \nabla_w L, \quad b \leftarrow b - \eta \frac{\partial L}{\partial b}$$

because $\nabla L$ points in the steepest increase direction.

## "Neural Net" View (Backprop Lens)

This is backprop in 2 local steps:

1. $L = \frac{1}{n}\sum e^2$
   $$\frac{\partial L}{\partial \hat{y}} = \frac{2}{n}(\hat{y} - y)$$

2. $\hat{y} = Xw + b$
   $$\frac{\partial \hat{y}}{\partial w} = X, \quad \frac{\partial \hat{y}}{\partial b} = 1$$
   
Chain:
$$\nabla_w L = X^\top \frac{\partial L}{\partial \hat{y}}, \quad \frac{\partial L}{\partial b} = \sum \frac{\partial L}{\partial \hat{y}}$$

<a id='proofs'></a>
## 3. Detailed Proofs

### Why $\frac{\partial e_i}{\partial w} = x_i$?

Because $e_i$ is **linear in $w$**.

Let $w=(w_1,\dots,w_d)$, $x_i=(x_{i1},\dots,x_{id})$. Then:

$$e_i = w^\top x_i + b - y_i = \sum_{j=1}^d w_j x_{ij} + b - y_i$$

Now take **partial derivative wrt one component** $w_k$:

$$\frac{\partial e_i}{\partial w_k} = \frac{\partial}{\partial w_k}\Big(\sum_{j=1}^d w_j x_{ij} + b - y_i\Big)$$

All terms with $j\neq k$ are constants wrt $w_k$, so vanish. Only $w_k x_{ik}$ remains:

$$\frac{\partial e_i}{\partial w_k} = x_{ik}$$

So the gradient wrt the whole vector $w$ is the vector of these partials:

$$\nabla_w e_i = \begin{bmatrix} \frac{\partial e_i}{\partial w_1} \\ \vdots \\ \frac{\partial e_i}{\partial w_d} \end{bmatrix} = \begin{bmatrix} x_{i1} \\ \vdots \\ x_{id} \end{bmatrix} = x_i$$

Then chain rule:

$$\frac{\partial}{\partial w}(e_i^2) = 2 e_i \frac{\partial e_i}{\partial w} = 2 e_i x_i$$

### Tiny 2D Example (By Hand)

$$e = w_1 x_1 + w_2 x_2 + b - y$$

$$\frac{\partial e}{\partial w_1} = x_1, \quad \frac{\partial e}{\partial w_2} = x_2$$

$$\Rightarrow \nabla_w e = (x_1, x_2) = x$$

<a id='implementation'></a>
## 4. Implementation

### Basic Functions

In [ ]:
def forward(X, w, b):
    return X @ w + b

In [ ]:
def mse(y_hat, y):
    return np.mean((y_hat - y) ** 2)

### Gradient Computation

In [ ]:
def compute_gradients(X, y, y_hat):
    n = X.shape[0]
    error = y_hat - y
    
    dw = (2/n) * X.T @ error
    db = (2/n) * np.sum(error)
    
    return dw, db

### Training Loop

In [ ]:
def train(X, y, learning_rate=0.01, epochs=1000):
    n, d = X.shape
    w = np.random.randn(d)
    b = 0.0
    losses = []
    
    for epoch in range(epochs):
        y_hat = forward(X, w, b)
        loss = mse(y_hat, y)
        losses.append(loss)
        
        dw, db = compute_gradients(X, y, y_hat)
        
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        if (epoch + 1) % 200 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.6f}")
    
    return w, b, losses

<a id='experiment'></a>
## 5. Experiment

### Generate Data

In [ ]:
np.random.seed(42)

n_samples = 100
n_features = 2

X = np.random.randn(n_samples, n_features)

In [ ]:
true_w = np.array([3.0, 2.0])
true_b = 1.0

In [ ]:
y = X @ true_w + true_b + 0.5 * np.random.randn(n_samples)

print(f"Data shape: X={X.shape}, y={y.shape}")
print(f"True parameters: w={true_w}, b={true_b}")

### Train Model

In [ ]:
w_learned, b_learned, losses = train(X, y, learning_rate=0.1, epochs=1000)

In [ ]:
print(f"\nLearned: w={w_learned}, b={b_learned:.4f}")
print(f"True: w={true_w}, b={true_b}")
print(f"Final loss: {losses[-1]:.6f}")

### Visualize Results

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training Loss Over Time')
plt.grid(True)
plt.show()

<a id='verification'></a>
## 6. Verification

Let's verify our gradient formulas manually.

In [ ]:
X_small = X[:5]
y_small = y[:5]

w_test = np.array([1.0, 1.0])
b_test = 0.0

In [ ]:
y_hat_test = forward(X_small, w_test, b_test)
error = y_hat_test - y_small

print("Error:", error)

In [ ]:
n = X_small.shape[0]
dw_manual = (2/n) * X_small.T @ error

print("Manual gradient dw:", dw_manual)

In [ ]:
dw_func, db_func = compute_gradients(X_small, y_small, y_hat_test)

print("Function gradient dw:", dw_func)
print("Match:", np.allclose(dw_manual, dw_func))

In [ ]:
db_manual = (2/n) * np.sum(error)

print("Manual gradient db:", db_manual)
print("Function gradient db:", db_func)
print("Match:", np.allclose(db_manual, db_func))

<a id='summary'></a>
## 7. Summary

We successfully implemented linear regression from first principles using:

1. **Mathematical derivation** of gradients:
   - $\nabla_w L = \frac{2}{n}X^\top(Xw + b - y)$
   - $\frac{\partial L}{\partial b} = \frac{2}{n}\sum(Xw + b - y)$

2. **Gradient descent** update:
   - $w \leftarrow w - \eta \nabla_w L$
   - $b \leftarrow b - \eta \frac{\partial L}{\partial b}$

3. **Clean, vectorized implementation**

### Key Insights:
- No activation function needed
- Linearity makes gradients straightforward
- Error $e_i = \hat{y}_i - y_i$ appears in both gradients
- Matrix operations efficiently compute all derivatives at once